In [1]:
# IMPORTS
from ipywidgets import HBox, VBox, Label, Layout, AppLayout, Output, Tab, Text
from IPython.display import display
import sys, os
import yaml
from pprint import pprint

# append parent of cwd to sys path to find modules
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

# surpress pygame version prompt
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"

# enable matplotlib widgets
%matplotlib widget

from src.app_batch import AppBatch as App
from src.datatypes.call_response_set import CallResponseSet
from dashboards.widgets import inputHeading, inputSelect, inputUpload, generationHeading, checkpointSelect, tempSlider, generationInfo, adaptationHeading, stepsSelect, startButton, applyGenerationSettingsButton, applyAdaptationSettingsButton, generationAmount, adaptationAmount, h2_heading

from src.io.plot import multitrack_pianoroll, multi_evaluation_bars
from src.io.output import play_button
from src.io.input import loadMidiFile

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
##########################
#C#    OUTPUT & APP    ###
##########################

log = Output()
output = Output()

summary_container = Output()
pianoroll_container = Output()
play_button_container = Output()
evaluation_container = Output()
meta_l_container = Output()
meta_m_container = Output()
meta_r_container = Output()
generation_info = Output()

# init
app = App(log=log)

with generation_info:
    display(generationInfo)

In [3]:
##########################
#M#   SELECTION DATA   ###
##########################

adaptation_operations = app.get_adaptation_operations()
generators = app.get_generators()
demo_melodies = app.get_demo_input()

checkpointSelect.options = [(g[0], generators.index(g)) for g in generators]
stepsSelect.options = adaptation_operations
inputSelect.options = [(d[0], demo_melodies.index(d)) for d in demo_melodies]

In [4]:
##########################
#C#     CONTROLLER     ###
##########################

def apply_generator_settings(b):
    generator = app.apply_generator_settings(checkpointSelect.value, tempSlider.value)
    generation_info.clear_output()
    with generation_info:
        print('Using: ' + generator)

def apply_adaptation_settings(b):
    app.apply_adaptation_settings(stepsSelect.value)

result = None

def run_adaptation(b):
    # reset output containers
    summary_container.clear_output()
    pianoroll_container.clear_output()
    play_button_container.clear_output()
    evaluation_container.clear_output()
    meta_l_container.clear_output()
    meta_m_container.clear_output()
    meta_r_container.clear_output()

    # TODO handle file upload
    input_path = demo_melodies[inputSelect.value][1]
    result = app.run(input_path, generationAmount.value, adaptationAmount.value, False)
    pprint(result)

    with summary_container:
        # TODO print metrics ? 
        pass

    # create avg similarity plot
    with summary_container: 
        display(h2_heading('Avg. Input-Similarity of all Generations/Adaptations (normalized):'))
    sim_data = [result['generation_set_avg_similarity']['normalized'], result['adaptation_avg_similarity']['normalized']]
    multi_evaluation_bars(sim_data, summary_container, [' Generations', 'Adaptations'])

    # create avg variance plot
    with summary_container: 
        display(h2_heading('Variance within Generations/Adaptations (normalized):'))
    sim_data = [result['generation_set_variance']['normalized'], result['adaptation_avg_variance']['normalized']]
    multi_evaluation_bars(sim_data, summary_container, ['Generations', 'Adaptations (avg.)'])


    display_details(result['generations'][0]['adaptations'][0])


def display_details(cr_set):
    # create pianoroll plots from results and print them
    midi_list = [cr_set.input_sequence.sequence, cr_set.generated_base_sequence.sequence, cr_set.output_sequence.sequence]
    name_list = ['Input', 'Generation', 'Result']
    multitrack_pianoroll(midi_list, name_list, pianoroll_container)

    # display play buttons for audio
    with play_button_container: 
        display(h2_heading('Input:'))
    play_button(midi_list[0], play_button_container)
    with play_button_container: 
        display(h2_heading('Generation:'))    
    play_button(midi_list[1], play_button_container)
    with play_button_container: 
        display(h2_heading('Adaptation:'))
    play_button(midi_list[2], play_button_container)

    # create evaluation plots
    eval_data = [cr_set.generation_similarity['normalized'], cr_set.output_similarity['normalized']]
    multi_evaluation_bars(eval_data, evaluation_container, ['Generated Sequence', 'Adapted Sequence'])

    # print meta data
    with meta_container:
        print('Input Analysis:')
        pprint(cr_set.input_sequence.analysis)
    with meta_m_container:
        print('Gen Analysis:')
        pprint(cr_set.output_sequence.analysis)
    with meta_r_container:
        print('Adaptation Meta:')
        pprint(cr_set.output_sequence.meta)



startButton.on_click(run_adaptation)
applyGenerationSettingsButton.on_click(apply_generator_settings)
applyAdaptationSettingsButton.on_click(apply_adaptation_settings)

In [5]:
##########################
#V#  SETTINGS SIDEBAR  ###
##########################

side_bar_layout = Layout(border='1px solid grey', padding='10px')

settings_box = VBox([
        generationHeading,
        HBox([Label('Model & Checkpoint:'), checkpointSelect]),
        HBox([Label('Temperature:'), tempSlider]),
        generation_info,
        applyGenerationSettingsButton,
        adaptationHeading,
        HBox([Label('Steps:'), stepsSelect]),
        applyAdaptationSettingsButton,
        inputHeading,
        inputSelect,
        HBox([Label('Or upload own file:'), inputUpload]),
        HBox([Label('Amount of generations:'), generationAmount]),
        HBox([Label('Amount of adaptations per generation:'), adaptationAmount]),
        startButton,
    ], layout = side_bar_layout)

In [17]:
##########################
#V#    RESULTS AREA    ###
##########################

main_layout = Layout(padding='10px')

# AVG & SUMMARY
summary_box = summary_container

# DETAILS
pianorolls = HBox([pianoroll_container, play_button_container])
meta = HBox([meta_l_container, meta_m_container, meta_r_container])
details_box = VBox([pianorolls, evaluation_container, meta, output], layout=main_layout)

In [18]:
##########################
#V#        TABS        ###
##########################

tabs = Tab()
tabs.children = [summary_box, details_box]
tabs.set_title(0, "Summary & Statistics")
tabs.set_title(1, "Detailed Results")

In [19]:
##########################
#V#     APP LAYOUT     ###
##########################

AppLayout(left_sidebar=settings_box,
          center=tabs,
          header=None,
          footer=log,
          right_sidebar=None,
          pane_widths=[1,2,0],
          pane_heights=[0,4,1])

AppLayout(children=(Output(layout=Layout(grid_area='footer'), outputs=({'name': 'stdout', 'text': 'Generating …

In [20]:
run_adaptation(startButton)  

s_transition_matrix': 0.0,
                                                             'pitch_count': 0.0,
                                                             'pitch_range': 0.0}},
                  'adaptations': [<src.datatypes.call_response_set.CallResponseSet object at 0x0000018010364688>,
                  'gen_data': <src.datatypes.melody_data.MelodyData object at 0x000001801046CC48>},
                 {'adaptation_set_avg_similarity': {'absolute': {'avg_ioi': 0.3990384615384616,
                                                                 'avg_pitch_interval': 0.24038461538461542,
                                                                 'note_count': 5.0,
                                                                 'note_length_histogram': 0.5167270098621596,
                                                                 'note_length_transition_matrix': 6.324555320336758,
                                                                 'pitch_class_hi

HTML(value='<h2>Generation:<h2>', placeholder='Generation:')

Input Analysis:
{'key': <music21.key.Key of C major>,
 'note_offsets_per_beat': [[0.0], [0.0], [0.0], [0.0]]}
